In [38]:
import numpy as np
import pandas as pd
from osgeo import gdal
from osgeo import osr
import os
import time

In [39]:
class Dataset:
    def __init__(self, in_file):
        self.in_file = in_file  # Tiff或者ENVI文件

        dataset = gdal.Open(self.in_file)
        self.XSize = dataset.RasterXSize  # 网格的X轴像素数量
        self.YSize = dataset.RasterYSize  # 网格的Y轴像素数量
        self.Bands = dataset.RasterCount  # 波段数
        self.GeoTransform = dataset.GetGeoTransform()  # 投影转换信息
        self.ProjectionInfo = dataset.GetProjection()  # 投影信息
    
    def get_Trans_Proj(self):
        #获取经纬度信息
        gtf = self.GeoTransform
        pro = self.ProjectionInfo
        xSize=self.XSize
        ySize=self.YSize
        Bands=self.Bands
        return [gtf, pro,xSize,ySize,Bands]
    
    def get_data(self):
        #band: 读取第几个通道的数据
        dataset = gdal.Open(self.in_file)
        data = dataset.ReadAsArray(0,0,self.XSize,self.YSize)
        
        return data

In [40]:
dir_path = r"D:\Desktop"
data=Dataset(os.path.join(dir_path, "GPP_make_thread.tif"))
dataset=data.get_data( ) 

In [41]:
# dataset=dataset[:,2::]
dataset.shape

(365, 630, 1439)

In [43]:
sum_data=np.mean(dataset,axis=0).astype(np.float32)
sum_data.shape

(630, 1439)

In [44]:
start=time.time()

Inform=data.get_Trans_Proj()
# [gtf, pro,xSize,ySize,Bands]
output_raster = gdal.GetDriverByName('GTiff').Create('D:\Desktop\mean.tif',Inform[2], Inform[3], 1 ,gdal.GDT_Float32)
output_raster.SetGeoTransform(Inform[0])
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)

output_raster.SetProjection(srs.ExportToWkt())

output_raster.GetRasterBand(1).WriteArray(sum_data)
output_raster.GetRasterBand(1).SetDescription("mean")
output_raster=None
print("total time:",time.time()-start)  

total time: 0.010971784591674805
